# Cleaning PubChemLite (Feb 2025)

In [23]:
import pandas as pd
import numpy as np

import rdkit
from rdkit import Chem, rdBase, RDLogger, DataStructs
RDLogger.DisableLog('rdApp.*')
from rdkit.Chem import AllChem, Draw, inchi, rdDepictor, PandasTools, SaltRemover, Descriptors
from rdkit.Chem.Draw import IPythonConsole, rdMolDraw2D
from rdkit.Chem.MolStandardize import rdMolStandardize

import pickle

In [24]:
pubchemlite_orig = pd.read_csv('PubChemLite_CCSbase_20250228.csv')

pubchemlite = pubchemlite_orig[['SMILES', 'InChIKey', 'CompoundName', 'Identifier', 'XLogP', 'MonoisotopicMass']]
pubchemlite.rename(columns={'Identifier': 'pubchem_cid',
                                   'CompoundName':'name',
                                   'MonoisotopicMass':'monoisotopic_mass_pubchem',
                                   'InChIKey':'InChIKey_pubchem',
                                   'SMILES':'SMILES_pubchem',
                                   'XLogP':'XLogP_pubchem'}, inplace=True)
pubchemlite

/var/folders/0w/cl6vcbp52mjfxw4v9_2l1_780000gn/T/ipykernel_76559/3596150552.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pubchemlite.rename(columns={'Identifier': 'pubchem_cid',


,SMILES_pubchem,InChIKey_pubchem,name,pubchem_cid,XLogP_pubchem,monoisotopic_mass_pubchem
0,C1=CC(C(C(=C1)C(=O)O)O)O,INCSWYKICIYAHB-UHFFFAOYSA-N,"5,6-dihydroxycyclohexa-1,3-diene-1-carboxylic ...",3,-0.3,156.042259
1,CC(CN)O,HXKKHQJGJAFBHI-UHFFFAOYSA-N,1-aminopropan-2-ol,4,-1.0,75.068414
2,C(C(=O)COP(=O)(O)O)N,HIQNVODXENYOFK-UHFFFAOYSA-N,(3-amino-2-oxopropyl) dihydrogen phosphate,5,-5.0,169.014009
3,C1=CC(=C(C=C1[N+](=O)[O-])[N+](=O)[O-])Cl,VYZAHLCBVHPDDF-UHFFFAOYSA-N,"1-chloro-2,4-dinitrobenzene",6,2.3,201.978134
4,CCC(C)(C(C(=O)O)O)O,PDGXJDXVGMHUIR-UHFFFAOYSA-N,"2,3-dihydroxy-3-methylpentanoic acid",8,-0.4,148.073559
...,...,...,...,...,...,...
408805,C1=CC(=C(C=C1NC(=O)NC2=C(C=C(C=C2)Cl)C3=C(C=CC...,CAWJHCNZWVFAPL-UHFFFAOYSA-N,"1-[4-chloro-2-[6-chloro-3-(2,4-dichlorophenoxy...",172643734,9.1,599.913558
408806,B1(OC(C(O1)(C)C)(C)C)C2=CC(=CN=C2)OC3COC3,VLOPGBXDLHSTBE-UHFFFAOYSA-N,"3-(oxetan-3-yloxy)-5-(4,4,5,5-tetramethyl-1,3,...",172644635,NaN,277.148538
408807,B1(OC(C(O1)(C)C)(C)C)C2=C(C=C(C=C2)C3COC3)C#N,YMEYBJPAQUDGPW-UHFFFAOYSA-N,"5-(oxetan-3-yl)-2-(4,4,5,5-tetramethyl-1,3,2-d...",172645001,NaN,285.153624
408808,B1(OC(C(O1)(C)C)(C)C)C2=C(C=CC(=C2)C3COC3)C#N,IYBNKVGFHAWEDF-UHFFFAOYSA-N,"4-(oxetan-3-yl)-2-(4,4,5,5-tetramethyl-1,3,2-d...",172645595,NaN,285.153624


### Cleaning of the SMILES and convertion to ROMol objects
The InChIKey, new SMILES and InChIKey14 will be computed from the ROMol objects after desalting and uncharging has been done. 

In [25]:
PandasTools.AddMoleculeColumnToFrame(pubchemlite, smilesCol='SMILES_pubchem', molCol='ROMol')

pubchemlite.ROMol.isna().sum()

/opt/anaconda3/envs/MS2Tox_APGC/lib/python3.12/site-packages/rdkit/Chem/PandasTools.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame[molCol] = frame[smilesCol].map(Chem.MolFromSmiles)


np.int64(9)

In [26]:
pubchemlite[pubchemlite.ROMol.isna()==True]

,SMILES_pubchem,InChIKey_pubchem,name,pubchem_cid,XLogP_pubchem,monoisotopic_mass_pubchem,ROMol
13982,O=Cl(=O)(=O)F,XHFXMNZYIKFCPN-UHFFFAOYSA-N,perchloryl fluoride,24258,3.3,101.952000,None
14115,FBr(F)F,FQFKTKUFHWNTBN-UHFFFAOYSA-N,trifluoro-lambda3-bromane,24594,2.6,135.913550,None
14117,FBr(F)(F)(F)F,XHVUVQAANZKEKF-UHFFFAOYSA-N,pentafluoro-lambda5-bromane,24606,3.7,173.910350,None
14137,FCl(F)F,JOHWNGGYGAVMGU-UHFFFAOYSA-N,trifluoro-lambda3-chlorane,24637,2.5,91.964062,None
32298,FCl(F)(F)(F)F,KNSWNNXPAWSACI-UHFFFAOYSA-N,pentafluoro-lambda5-chlorane,61654,3.6,129.960869,None
46001,FI(F)(F)(F)(F)(F)F,XRURPHMPXJDCOO-UHFFFAOYSA-N,heptafluoro-lambda7-iodane,85645,4.7,259.893290,None
162802,C1=CC=C2C(=C1)O[Si-2]34(O2)(OC5=CC=CC=C5O3)OC6...,JIICOOGKGWBPSB-UHFFFAOYSA-N,NaN,3470663,NaN,352.040315,None
266999,CC1=CC(=[N+](C(=C1)C)[Br-][N+]2=C(C=C(C=C2C)C)C)C,DWDPGQBPOJKMQW-UHFFFAOYSA-N,"2,4,6-trimethyl-1-(2,4,6-trimethylpyridin-1-iu...",44629788,5.5,321.096640,None
277611,C1=CC=C(C=C1)[Si](Cl)(Cl)[Cl-]Cl,MIUKAZMTBNWUDP-UHFFFAOYSA-N,dichloro-chlorochloranuidyl-phenylsilane,53400412,NaN,244.891462,None


In [27]:
pubchemlite = pubchemlite.dropna(subset=['ROMol'])

In [28]:
salts = pubchemlite.SMILES_pubchem.str.contains(r'\.')

salts.sum() #Is 0 so no salt removal needed

np.int64(0)

In [29]:
def contains_carbon(mol):
    try:
        atomic_no = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
        return bool(6 in atomic_no)
    except:
        return np.nan

pubchemlite['contains_carbon'] = pubchemlite.ROMol.apply(lambda x: contains_carbon(x))
pubchemlite = pubchemlite[pubchemlite.contains_carbon == True]
pubchemlite = pubchemlite.drop(columns=['contains_carbon'])

/var/folders/0w/cl6vcbp52mjfxw4v9_2l1_780000gn/T/ipykernel_76559/1547849159.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pubchemlite['contains_carbon'] = pubchemlite.ROMol.apply(lambda x: contains_carbon(x))


In [30]:
uncharger = rdMolStandardize.Uncharger()  # neutralize the molecule (if possible)
pubchemlite['ROMol'] = pubchemlite.ROMol.apply(lambda x: uncharger.uncharge(x))

# def neutralize_atoms(mol):
#     try:
#         pattern = Chem.MolFromSmarts("[+1!h0!$([*]~[-1,-2,-3,-4]),-1!$([*]~[+1,+2,+3,+4])]")
#         at_matches = mol.GetSubstructMatches(pattern)
#         at_matches_list = [y[0] for y in at_matches]
#         if len(at_matches_list) > 0:
#             for at_idx in at_matches_list:
#                 atom = mol.GetAtomWithIdx(at_idx)
#                 chg = atom.GetFormalCharge()
#                 hcount = atom.GetTotalNumHs()
#                 atom.SetFormalCharge(0)
#                 atom.SetNumExplicitHs(hcount - chg)
#                 atom.UpdatePropertyCache()
#         return mol
#     except:
#         return mol

#pubchemlite_subset['ROMol'] = pubchemlite_subset.ROMol.apply(lambda x: neutralize_atoms(x))
pubchemlite.ROMol.apply(lambda x: Chem.RemoveStereochemistry(x)) 
pubchemlite['SMILES'] = pubchemlite.ROMol.apply(lambda x: Chem.MolToSmiles(x))
pubchemlite['InChIKey'] = pubchemlite.ROMol.apply(lambda x: inchi.MolToInchiKey(x))
pubchemlite['InChIKey14'] = pubchemlite.InChIKey.apply(lambda x: x.split('-')[0])

In [31]:
pubchemlite_dupl = pubchemlite[pubchemlite.duplicated(subset=['InChIKey', 'SMILES'], keep=False)]
pubchemlite_dupl = pubchemlite_dupl.sort_values(by='InChIKey')
pubchemlite_dupl

,SMILES_pubchem,InChIKey_pubchem,name,pubchem_cid,XLogP_pubchem,monoisotopic_mass_pubchem,ROMol,SMILES,InChIKey,InChIKey14
212955,C1=CC=C2C(=C1)N=NN2[O-],VGZMHTGJWOPMTH-UHFFFAOYSA-N,1-oxidobenzotriazole,11805229,0.7,134.035437,<rdkit.Chem.rdchem.Mol object at 0x1739d5770>,On1nnc2ccccc21,ASOKPJOREAFHNY-UHFFFAOYSA-N,ASOKPJOREAFHNY
40303,C1=CC=C2C(=C1)N=NN2O,ASOKPJOREAFHNY-UHFFFAOYSA-N,1-hydroxybenzotriazole,75771,1.2,135.043262,<rdkit.Chem.rdchem.Mol object at 0x1693cf450>,On1nnc2ccccc21,ASOKPJOREAFHNY-UHFFFAOYSA-N,ASOKPJOREAFHNY
72524,C=C1C=NC2=CC=CC=C12,BCNUXXXHEIUHJB-UHFFFAOYSA-N,3-methylideneindole,170404,1.7,129.057849,<rdkit.Chem.rdchem.Mol object at 0x14e988eb0>,C=C1C=Nc2ccccc21,BCNUXXXHEIUHJB-UHFFFAOYSA-N,BCNUXXXHEIUHJB
327565,[CH+]=C1C=NC2=CC=CC=C12,NSQIQDGIPQVASR-UHFFFAOYSA-N,3-methylideneindole,123131584,1.7,128.050024,<rdkit.Chem.rdchem.Mol object at 0x176c96500>,C=C1C=Nc2ccccc21,BCNUXXXHEIUHJB-UHFFFAOYSA-N,BCNUXXXHEIUHJB
259091,C1=CC=C2C(=C1)N(C(=O)C(O2)O)[O-],QUBXBHWFQCVXQY-UHFFFAOYSA-N,"2-hydroxy-4-oxido-1,4-benzoxazin-3-one",25203008,-0.1,180.029683,<rdkit.Chem.rdchem.Mol object at 0x14d15a0a0>,O=C1C(O)Oc2ccccc2N1O,COVOPZQGJGUPEY-UHFFFAOYSA-N,COVOPZQGJGUPEY
15950,C1=CC=C2C(=C1)N(C(=O)C(O2)O)O,COVOPZQGJGUPEY-UHFFFAOYSA-N,"2,4-dihydroxy-1,4-benzoxazin-3-one",28495,0.0,181.037508,<rdkit.Chem.rdchem.Mol object at 0x14d8959a0>,O=C1C(O)Oc2ccccc2N1O,COVOPZQGJGUPEY-UHFFFAOYSA-N,COVOPZQGJGUPEY
393497,CC(=C[CH+]C(=O)O)C,XZWWIRTVQPJTML-UHFFFAOYSA-O,4-methylpent-3-enoic acid,135939139,1.4,113.060255,<rdkit.Chem.rdchem.Mol object at 0x1499d4f90>,CC(C)=CCC(=O)O,CQJHAULYLJXJNL-UHFFFAOYSA-N,CQJHAULYLJXJNL
35975,CC(=CCC(=O)O)C,CQJHAULYLJXJNL-UHFFFAOYSA-N,4-methylpent-3-enoic acid,68158,1.4,114.068080,<rdkit.Chem.rdchem.Mol object at 0x300f18200>,CC(C)=CCC(=O)O,CQJHAULYLJXJNL-UHFFFAOYSA-N,CQJHAULYLJXJNL
320975,CN(C)CCCN(O)S(=O)(=O)CCC(C(C(C(C(C(F)(F)F)(F)F...,CUJCSBFFGKTHSK-UHFFFAOYSA-N,"N-[3-(dimethylamino)propyl]-3,3,4,4,5,5,6,6,7,...",102055960,4.2,528.075230,<rdkit.Chem.rdchem.Mol object at 0x13e478890>,CN(C)CCCN(O)S(=O)(=O)CCC(F)(F)C(F)(F)C(F)(F)C(...,CUJCSBFFGKTHSK-UHFFFAOYSA-N,CUJCSBFFGKTHSK
69861,CN(C)CCCN([O-])S(=O)(=O)CCC(C(C(C(C(C(F)(F)F)(...,COSQFXYPUSWVAZ-UHFFFAOYSA-N,"N-[3-(dimethylamino)propyl]-3,3,4,4,5,5,6,6,7,...",157338,4.2,527.067405,<rdkit.Chem.rdchem.Mol object at 0x14e9d4040>,CN(C)CCCN(O)S(=O)(=O)CCC(F)(F)C(F)(F)C(F)(F)C(...,CUJCSBFFGKTHSK-UHFFFAOYSA-N,CUJCSBFFGKTHSK


In [32]:
pubchemlite_dupl['monoisotopic_mass_new'] = pubchemlite_dupl.ROMol.apply(lambda x: Descriptors.ExactMolWt(x)) #calculates the monoisotopic mass of the molecule to determine which represenetation is the correct one
pubchemlite_dupl['monoisotopic_mass_diff'] = abs(pubchemlite_dupl.monoisotopic_mass_new - pubchemlite_dupl.monoisotopic_mass_pubchem) #gives absolute mass difference

pubchemlite_dupl_removed = pubchemlite_dupl[~(pubchemlite_dupl.monoisotopic_mass_diff > 1)] #Removes the duplicates with a mass difference of more than 1 Da
pubchemlite_dupl_removed = pubchemlite_dupl_removed.drop(columns=['monoisotopic_mass_new', 'monoisotopic_mass_diff']) #drops the columns concerning the mass difference
pubchemlite_dupl_removed

,SMILES_pubchem,InChIKey_pubchem,name,pubchem_cid,XLogP_pubchem,monoisotopic_mass_pubchem,ROMol,SMILES,InChIKey,InChIKey14
40303,C1=CC=C2C(=C1)N=NN2O,ASOKPJOREAFHNY-UHFFFAOYSA-N,1-hydroxybenzotriazole,75771,1.2,135.043262,<rdkit.Chem.rdchem.Mol object at 0x1693cf450>,On1nnc2ccccc21,ASOKPJOREAFHNY-UHFFFAOYSA-N,ASOKPJOREAFHNY
72524,C=C1C=NC2=CC=CC=C12,BCNUXXXHEIUHJB-UHFFFAOYSA-N,3-methylideneindole,170404,1.7,129.057849,<rdkit.Chem.rdchem.Mol object at 0x14e988eb0>,C=C1C=Nc2ccccc21,BCNUXXXHEIUHJB-UHFFFAOYSA-N,BCNUXXXHEIUHJB
15950,C1=CC=C2C(=C1)N(C(=O)C(O2)O)O,COVOPZQGJGUPEY-UHFFFAOYSA-N,"2,4-dihydroxy-1,4-benzoxazin-3-one",28495,0.0,181.037508,<rdkit.Chem.rdchem.Mol object at 0x14d8959a0>,O=C1C(O)Oc2ccccc2N1O,COVOPZQGJGUPEY-UHFFFAOYSA-N,COVOPZQGJGUPEY
35975,CC(=CCC(=O)O)C,CQJHAULYLJXJNL-UHFFFAOYSA-N,4-methylpent-3-enoic acid,68158,1.4,114.068080,<rdkit.Chem.rdchem.Mol object at 0x300f18200>,CC(C)=CCC(=O)O,CQJHAULYLJXJNL-UHFFFAOYSA-N,CQJHAULYLJXJNL
320975,CN(C)CCCN(O)S(=O)(=O)CCC(C(C(C(C(C(F)(F)F)(F)F...,CUJCSBFFGKTHSK-UHFFFAOYSA-N,"N-[3-(dimethylamino)propyl]-3,3,4,4,5,5,6,6,7,...",102055960,4.2,528.075230,<rdkit.Chem.rdchem.Mol object at 0x13e478890>,CN(C)CCCN(O)S(=O)(=O)CCC(F)(F)C(F)(F)C(F)(F)C(...,CUJCSBFFGKTHSK-UHFFFAOYSA-N,CUJCSBFFGKTHSK
318649,C/C(=C/C(=O)N(CCCC1C(=O)NC(C(=O)NC(C(=O)NC(C(=...,DKSPATNIHZSGOP-YZSKCGIOSA-N,(Z)-5-[3-[5-[3-[[(E)-4-carboxy-3-methylbut-2-e...,91819949,-3.2,999.403285,<rdkit.Chem.rdchem.Mol object at 0x13e4c46d0>,CC(=CC(=O)N(O)CCCC1NC(=O)CNC(=O)C(CO)NC(=O)C(C...,DKSPATNIHZSGOP-UHFFFAOYSA-N,DKSPATNIHZSGOP
830,COC1=CC2=C(C=C1)N(C(=O)C(O2)O)O,GDNZNIJPBQATCZ-UHFFFAOYSA-N,"2,4-dihydroxy-7-methoxy-1,4-benzoxazin-3-one",2358,0.0,211.048072,<rdkit.Chem.rdchem.Mol object at 0x1763025e0>,COc1ccc2c(c1)OC(O)C(=O)N2O,GDNZNIJPBQATCZ-UHFFFAOYSA-N,GDNZNIJPBQATCZ
37218,C1=CC=C(C=C1)PC2=CC=CC=C2,GPAYUJZHTULNBE-UHFFFAOYSA-N,diphenylphosphane,70017,3.0,186.059837,<rdkit.Chem.rdchem.Mol object at 0x1693965e0>,c1ccc(Pc2ccccc2)cc1,GPAYUJZHTULNBE-UHFFFAOYSA-N,GPAYUJZHTULNBE
161598,C1=CC=C(C=C1)CC(=S)NO,IHTJGIKQNHDTSX-UHFFFAOYSA-N,N-hydroxy-2-phenylethanethioamide,3295299,1.2,167.040485,<rdkit.Chem.rdchem.Mol object at 0x14a3acd60>,ONC(=S)Cc1ccccc1,IHTJGIKQNHDTSX-UHFFFAOYSA-N,IHTJGIKQNHDTSX
70075,C1=CN(C(=N1)[N+](=O)[O-])CC(=O)NO,KAIQUJDTIFXCLP-UHFFFAOYSA-N,N-hydroxy-2-(2-nitroimidazol-1-yl)acetamide,159484,-1.1,186.038905,<rdkit.Chem.rdchem.Mol object at 0x14e9c2260>,O=C(Cn1ccnc1[N+](=O)[O-])NO,KAIQUJDTIFXCLP-UHFFFAOYSA-N,KAIQUJDTIFXCLP


In [ ]:
pubchemlite_no_dupl = pubchemlite[~pubchemlite.InChIKey.isin(pubchemlite_dupl.InChIKey)] #Remove any duplicates from the original dataframe

pubchemlite_std = pd.concat([pubchemlite_no_dupl, pubchemlite_dupl_removed], ignore_index=True).reset_index(drop=True) #removes the duplicates from the original dataframe and appends the new ones


In [50]:
pubchemlite_std.shape

(407967, 10)

In [ ]:
# with open('2025-04-09_PubChemLite_smiles_cleaned.pkl', 'wb') as f:
#       pickle.dump(pubchemlite, f)

### Standardization of the data using the PubChem StandardUtils

Divide into multiple batches to process over multiple evenings, and then add onto each other when they are all done. 

In [ ]:
with open('2025-04-09_PubChemLite_smiles_cleaned.pkl', 'rb') as f:
    pubchemlite = pickle.load(f)

In [ ]:
pubchemlite_batches = np.array_split(pubchemlite, 4)

/opt/anaconda3/envs/MS2Tox_APGC/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [5]:
pubchemlite_batches[0]

,SMILES,InChIKey,CompoundName,PubChem_cid,XLogP,MonoisotopicMass,ROMol,InChIKey14
0,O=C(O)C1=CC=CC(O)C1O,INCSWYKICIYAHB-UHFFFAOYSA-N,"5,6-dihydroxycyclohexa-1,3-diene-1-carboxylic ...",3,-0.3,156.042259,<rdkit.Chem.rdchem.Mol object at 0x169ff38d0>,INCSWYKICIYAHB
1,CC(O)CN,HXKKHQJGJAFBHI-UHFFFAOYSA-N,1-aminopropan-2-ol,4,-1.0,75.068414,<rdkit.Chem.rdchem.Mol object at 0x169ff3920>,HXKKHQJGJAFBHI
2,NCC(=O)COP(=O)(O)O,HIQNVODXENYOFK-UHFFFAOYSA-N,(3-amino-2-oxopropyl) dihydrogen phosphate,5,-5.0,169.014009,<rdkit.Chem.rdchem.Mol object at 0x169ff3970>,HIQNVODXENYOFK
3,O=[N+]([O-])c1ccc(Cl)c([N+](=O)[O-])c1,VYZAHLCBVHPDDF-UHFFFAOYSA-N,"1-chloro-2,4-dinitrobenzene",6,2.3,201.978134,<rdkit.Chem.rdchem.Mol object at 0x169ff39c0>,VYZAHLCBVHPDDF
4,CCC(C)(O)C(O)C(=O)O,PDGXJDXVGMHUIR-UHFFFAOYSA-N,"2,3-dihydroxy-3-methylpentanoic acid",8,-0.4,148.073559,<rdkit.Chem.rdchem.Mol object at 0x169ff3a10>,PDGXJDXVGMHUIR
...,...,...,...,...,...,...,...,...
101987,COC(=O)c1cnc(N)[nH]1,NDTWFWAWLBGFQI-UHFFFAOYSA-N,methyl 2-amino-1H-imidazole-5-carboxylate,588062,-0.1,141.053826,<rdkit.Chem.rdchem.Mol object at 0x32e3a74c0>,NDTWFWAWLBGFQI
101988,O=C(Cn1cncn1)c1ccc(F)cc1F,XCHRPVARHBCFMJ-UHFFFAOYSA-N,"1-(2,4-difluorophenyl)-2-(1,2,4-triazol-1-yl)e...",588080,1.7,223.055718,<rdkit.Chem.rdchem.Mol object at 0x32e3a7510>,XCHRPVARHBCFMJ
101989,O=C(Cl)c1ccc(F)cc1F,JSWRVDNTKPAJLB-UHFFFAOYSA-N,"2,4-difluorobenzoyl chloride",588081,2.6,175.984049,<rdkit.Chem.rdchem.Mol object at 0x32e3a7560>,JSWRVDNTKPAJLB
101990,O=C(Cl)c1cc(F)ccc1F,RLRUKKDFNWXXRT-UHFFFAOYSA-N,"2,5-difluorobenzoyl chloride",588082,2.6,175.984049,<rdkit.Chem.rdchem.Mol object at 0x32e3a75b0>,RLRUKKDFNWXXRT


In [41]:
pubchemlite_head = pubchemlite.head(1000)

In [43]:
from standardizeUtils.standardizeUtils import standardize_structure_with_pubchem
from standardizeUtils.standardizeUtils import standardize_structure_list_with_pubchem

import time
import requests  



def standardize_in_batches(smiles_list, initial_batch_size=1000):
    """
    Standardizes SMILES in batches, reducing batch size on failure.
    Input: SMILES list
    Output: Standardized SMILES list
    """
    total_smiles = len(smiles_list)
    processed_smiles = []
    idx = 0
    batch_size = initial_batch_size # Starting with 1000

    while idx < total_smiles:
        attempt_success = False
        current_batch_size = batch_size

        while current_batch_size >= 100:
            try:
                print(f"Processing batch {idx}:{idx + current_batch_size} (size={current_batch_size})")
                batch = smiles_list[idx : idx + current_batch_size]
                standardized_batch = standardize_structure_list_with_pubchem(batch, 'smiles')
                processed_smiles.extend(standardized_batch)
                idx += current_batch_size
                attempt_success = True
                break  

            except requests.exceptions.RequestException as e:
                print(f"Error occurred: {e}, reducing batch size...")
                current_batch_size //= 2  # Reduce batch size by half on failure > can be changed
                time.sleep(2)  

        if not attempt_success:
            print(f"Skipping batch {idx}:{idx + current_batch_size} due to repeated failures.")
            idx += current_batch_size  

    return processed_smiles

pubchemlite_head['std_SMILES'] = standardize_in_batches(pubchemlite_head.SMILES.to_list())

Processing batch 0:1000 (size=1000)


/var/folders/0w/cl6vcbp52mjfxw4v9_2l1_780000gn/T/ipykernel_76559/4207872608.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pubchemlite_head['std_SMILES'] = standardize_in_batches(pubchemlite_head.SMILES.to_list())


In [ ]:
pubchemlite_head[pubchemlite_head.SMILES_pubchem!=pubchemlite_head.std_SMILES][['SMILES_pubchem', 'std_SMILES']] # The SMILES from standardization and pubchem smiles might in some cases differ, standardization is therefore needed. 

,SMILES_pubchem,std_SMILES
237,CN1CC(=O)N=C1N,CN1CC(=O)NC1=N


In [14]:
#pubchemlite_batches[0]['std_SMILES'] = standardize_in_batches(pubchemlite_batches[0].SMILES.to_list())
#pubchemlite_batches[1]['std_SMILES'] = standardize_in_batches(pubchemlite_batches[1].SMILES.to_list())
pubchemlite_batches[2]['std_SMILES'] = standardize_in_batches(pubchemlite_batches[2].SMILES.to_list())
#pubchemlite_batches[3]['std_SMILES'] = standardize_in_batches(pubchemlite_batches[3].SMILES.to_list())

with open('2025-03-19_PubChemLite_std_smiles_batch3.pkl', 'wb') as f:
    pickle.dump(pubchemlite_batches[2], f)

Processing batch 0:1000 (size=1000)


The standardization request failed. The response contains the following "PCT-Status" tag: <PCT-Status value="data-error"/>
An exception of class InputDataError was thrown. Return 'None' for query "Fc1ccc([BH](F)(F)F)cc1".
The standardization request failed. The response contains the following "PCT-Status" tag: <PCT-Status value="data-error"/>
An exception of class InputDataError was thrown. Return 'None' for query "CCCCC#C[BH](F)(F)F".
The standardization request failed. The response contains the following "PCT-Status" tag: <PCT-Status value="data-error"/>
An exception of class InputDataError was thrown. Return 'None' for query "O=Cc1ccccc1[BH](F)(F)F".


Processing batch 1000:2000 (size=1000)


The standardization request failed. The response contains the following "PCT-Status" tag: <PCT-Status value="data-error"/>
An exception of class InputDataError was thrown. Return 'None' for query "CC(=O)O[BH](OC(C)=O)OC(C)=O".
The standardization request failed. The response contains the following "PCT-Status" tag: <PCT-Status value="data-error"/>
An exception of class InputDataError was thrown. Return 'None' for query "C[BH](C)(C)C".
The standardization request failed. The response contains the following "PCT-Status" tag: <PCT-Status value="data-error"/>
An exception of class InputDataError was thrown. Return 'None' for query "c1cnn([BH](n2cccn2)n2cccn2)c1".
The standardization request failed. The response contains the following "PCT-Status" tag: <PCT-Status value="data-error"/>
An exception of class InputDataError was thrown. Return 'None' for query "Cc1cc(C)n([BH](n2nc(C)cc2C)n2nc(C)cc2C)n1".
The standardization request failed. The response contains the following "PCT-Status" tag: <

Processing batch 2000:3000 (size=1000)


...

The standardization request failed. The response contains the following "PCT-Status" tag: <PCT-Status value="data-error"/>
An exception of class InputDataError was thrown. Return 'None' for query "CC(C)(C)OC(=O)N1CC([BH](F)(F)F)C1".
The standardization request failed. The response contains the following "PCT-Status" tag: <PCT-Status value="data-error"/>
An exception of class InputDataError was thrown. Return 'None' for query "F[BH](F)(F)C1COC1".
The standardization request failed. The response contains the following "PCT-Status" tag: <PCT-Status value="data-error"/>
An exception of class InputDataError was thrown. Return 'None' for query "CC(C)(C)OC(=O)N1CCC([BH](F)(F)F)CC1".
The standardization request failed. The response contains the following "PCT-Status" tag: <PCT-Status value="data-error"/>
An exception of class InputDataError was thrown. Return 'None' for query "CC(C)(C)OC(=O)N1CCC([BH](F)(F)F)C1".


Processing batch 99000:100000 (size=1000)


The standardization request failed. The response contains the following "PCT-Status" tag: <PCT-Status value="data-error"/>
An exception of class InputDataError was thrown. Return 'None' for query "F[BH](F)(F)CCc1ccccn1".


Processing batch 100000:101000 (size=1000)
Processing batch 101000:102000 (size=1000)


The standardization request failed. The response contains the following "PCT-Status" tag: <PCT-Status value="data-error"/>
An exception of class InputDataError was thrown. Return 'None' for query "CC(C)C(N)C(=O)OC(C)C1CC=CCCC(O)C(C)(C)C2CCC(C)C3(O2)O[BH]24OC(C(=O)O1)C1(OC(CCC1C)C(C)(C)C(O)CCCC1CC(OC(=O)C3O2)C(C)O1)O4".
The standardization request failed. The response contains the following "PCT-Status" tag: <PCT-Status value="data-error"/>
An exception of class InputDataError was thrown. Return 'None' for query "COCCNC(=O)c1ccc([BH](F)(F)F)cc1".
The standardization request failed. The response contains the following "PCT-Status" tag: <PCT-Status value="data-error"/>
An exception of class InputDataError was thrown. Return 'None' for query "COCCNC(=O)c1cccc([BH](F)(F)F)c1".
The standardization request failed. The response contains the following "PCT-Status" tag: <PCT-Status value="data-error"/>
An exception of class InputDataError was thrown. Return 'None' for query "O=C(c1ccc([BH](F)(F)

In [12]:
pubchemlite_batches[1][pubchemlite_batches[1].std_SMILES.isna()]

,SMILES,InChIKey,CompoundName,PubChem_cid,XLogP,MonoisotopicMass,ROMol,InChIKey14,std_SMILES
118475,[2H][BH]([2H])([2H])C#N,BBWWRPHGUGQYTM-BMSJAHLVSA-N,cyano(trideuterio)boranuide,2724022,NaN,43.054685,<rdkit.Chem.rdchem.Mol object at 0x338242980>,BBWWRPHGUGQYTM,None
125203,FC(F)(F)c1cc([BH](c2cc(C(F)(F)F)cc(C(F)(F)F)c2...,LSJLFVQOCPUFMW-UHFFFAOYSA-N,"tetrakis[3,5-bis(trifluoromethyl)phenyl]boranuide",2778201,NaN,863.064881,<rdkit.Chem.rdchem.Mol object at 0x33ae56840>,LSJLFVQOCPUFMW,None
125930,F[BH](F)(F)c1ccccc1,TWOKLPOPVVGTTR-UHFFFAOYSA-N,trifluoro(phenyl)boranuide,2782845,NaN,145.043640,<rdkit.Chem.rdchem.Mol object at 0x33b610c70>,TWOKLPOPVVGTTR,None
125931,COc1ccccc1[BH](F)(F)F,UMTMKESCTWSUTC-UHFFFAOYSA-N,trifluoro-(2-methoxyphenyl)boranuide,2782847,NaN,175.054204,<rdkit.Chem.rdchem.Mol object at 0x33b610cc0>,UMTMKESCTWSUTC,None
125932,Cc1ccccc1[BH](F)(F)F,ZMZPFKAYZNLWMF-UHFFFAOYSA-N,trifluoro-(2-methylphenyl)boranuide,2782849,NaN,159.059290,<rdkit.Chem.rdchem.Mol object at 0x33b610d10>,ZMZPFKAYZNLWMF,None
125933,COc1ccc([BH](F)(F)F)cc1F,ZRSQBNZILRUUOM-UHFFFAOYSA-N,trifluoro-(3-fluoro-4-methoxyphenyl)boranuide,2782851,NaN,193.044783,<rdkit.Chem.rdchem.Mol object at 0x33b610d60>,ZRSQBNZILRUUOM,None
125934,COc1ccc([BH](F)(F)F)cc1,ZBWDBWABYIOLLG-UHFFFAOYSA-N,trifluoro-(4-methoxyphenyl)boranuide,2782853,NaN,175.054204,<rdkit.Chem.rdchem.Mol object at 0x33b610db0>,ZBWDBWABYIOLLG,None
125935,CS(=O)(=O)c1ccccc1[BH](F)(F)F,VUPYXBLIMCNRBG-UHFFFAOYSA-N,trifluoro-(2-methylsulfonylphenyl)boranuide,2782857,NaN,223.021190,<rdkit.Chem.rdchem.Mol object at 0x33b610e00>,VUPYXBLIMCNRBG,None
125936,CS(=O)(=O)Nc1cccc([BH](F)(F)F)c1,ZKTVPPQABWFBNG-UHFFFAOYSA-N,trifluoro-[3-(methanesulfonamido)phenyl]boranuide,2782859,NaN,238.032089,<rdkit.Chem.rdchem.Mol object at 0x33b610e50>,ZKTVPPQABWFBNG,None
125937,F[BH](F)(F)c1cccc(Cl)c1,UBLNYMJDVJUGMF-UHFFFAOYSA-N,(3-chlorophenyl)-trifluoroboranuide,2782861,NaN,179.004668,<rdkit.Chem.rdchem.Mol object at 0x33b610ea0>,UBLNYMJDVJUGMF,None


### Connection to terminal to make fingerprints from the SMILES

Will connect to the SIRIUS CLI to use the fingerprinter function, that will convert the standardized SMILES to fingerprints. 

Code made by Assist. Prof. Ida Rahu. 

In [ ]:
import subprocess

import pandas as pd
import numpy as np

import requests

email = 'ellinor.samuelsson-hoppe@su.se'
password = 'Canmake123321?'

#subprocess.run('/Applications/sirius.app/Contents/MacOS/sirius') #added to work with sirius command

process = subprocess.Popen(['/Applications/sirius.app/Contents/MacOS/sirius', 'login', '-u', email, '-p'], 
                           stdout=subprocess.PIPE, 
                           stderr=subprocess.PIPE,
                           stdin=subprocess.PIPE,
                           text=True)

process.communicate(input=password + '\n')

stdout, stderr = process.communicate()
print('STDOUT:\n', stdout)
print('STDERR:\n', stderr)

path = '/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Ellinor - Master thesis/Data/PubChemLite/'

smiles_file = path + '\\.tsv'
output_file = path + '\\.tsv'
fp_version_file = path + '\\.tsv'
charge = 1 # 1 for ESI+/ -1 for ESI-
command = ['/Applications/sirius.app/Contents/MacOS/sirius', '-i', smiles_file, 'fingerprinter', '--charge', str(charge), '-o', output_file, '-v', fp_version_file]

result = subprocess.run(command, capture_output=True, text=True)

print('STDOUT:\n', result.stdout)
print('STDERR:\n', result.stderr)

STDOUT:
 Enter value for --password (Console password input.): 
Active Subscription is: 'sub|adc29910-cf73-11ec-9d64-0242ac120002 - Academic License'.
Login successful!

STDERR:
 Mar 12, 2025 3:18:15 PM org.apache.commons.beanutils.FluentPropertyBeanIntrospector introspect
INFO: Error when creating PropertyDescriptor for public final void org.apache.commons.configuration2.AbstractConfiguration.setProperty(java.lang.String,java.lang.Object)! Ignoring this property.
Mar 12, 2025 3:18:16 PM de.unijena.bioinf.ms.frontend.core.ApplicationCore <clinit>
INFO: Sirius Workspace Successfull initialized at: /Users/elli/.sirius-5.8
Mar 12, 2025 3:18:16 PM de.unijena.bioinf.ms.frontend.core.ApplicationCore <clinit>
INFO: You run SIRIUS 5.8.7-SNAPSHOT on mac os x_x86_64
Mar 12, 2025 3:18:16 PM de.unijena.bioinf.ms.frontend.core.ApplicationCore <clinit>
INFO: You run SIRIUS in 'CLI' mode.
Mar 12, 2025 3:18:16 PM de.unijena.bioinf.ms.frontend.core.ApplicationCore <clinit>
INFO: Sirius was compiled wit

In [ ]:
with open('2025-03-12_PubChemLite_cleaned_smiles.pkl', 'wb') as f:
    pickle.dump(pubchemlite_subset, f)